# xmon1 <-> xmon2

In [71]:
import numpy as np
import numpy.linalg as alg
import scqubits as scq

In [72]:
hbar = 1.054e-34
h = hbar*2*np.pi
qe = 1.609e-19
flux_quant = h/2/qe

# Unit conversions
MHz = 10.0**(-3)
GHz = 1.0
kHz = 10.0**(-6)
us = 10.0**3
ns = 1.0

In [73]:
# Q3D VALUES
# All in SI units
Cjs = np.array([100.282, 82.822])*1e-15 # fF -> F
Cab = np.array([2.64])*1e-15 # [C12] fF -> F
Crqs = np.array([10.3, 7.744])*1e-15 # fF -> F
Cds = np.array([64, 56])*1e-18 # aF -> F

# HFSS VALUES
res_freqs = np.array([6.047, 7.252]) # GHz
# res_freqs = np.array([5.4, 6.4]) # GHz
# res_freqs = np.array([6.5, 7.7]) # GHz
Q_res = np.array([np.inf]*len(Cjs))
Q_drives = np.array([np.inf]*len(Cjs))

# GET ZO FROM CPW MATHEMATICA
Zo = 52.7066 # Ohm

# Ej_targets = np.array([13.51, 17.88])*1e9 # GHz -> Hz
Ej_targets = np.array([9.54, 12.52])*1e9 # GHz -> Hz
# Ej_targets = np.array([17.48, 21.85])*1e9 # GHz -> Hz
Ljs = flux_quant**2/h/4/np.pi**2/Ej_targets # H

c_matrix = np.array([
    [Cjs[0]+Cab[0], -Cab[0]],
    [-Cab[0],    Cjs[1]+Cab[0]],
])

inv_l_matrix = np.diag(1/Ljs)

print('Check symmetric:', np.allclose(c_matrix, c_matrix.T), np.allclose(inv_l_matrix, inv_l_matrix.T))

Check symmetric: True True


# BEGIN ANALYSIS

In [74]:
inv_c_matrix = alg.inv(c_matrix)

evals_c, u1 = alg.eigh(inv_c_matrix)
u1 = np.array([u1[i]/np.sqrt(evals_c[i]) for i in range(len(evals_c))])
evals, evecs = alg.eigh(u1.transpose() @ inv_l_matrix @ u1)
print('Modes (cols are evecs)\n', np.around(evecs, 3))

# ======== IDK WHY BUT THIS DOESN'T WORK ========
# Look for col with max in Q1_NODE -> use q_modes to reference in EC, EJ
# q_modes = np.array([np.argmax(np.abs(evecs[i,:])) for i in range(4)])
# print('Qubit modes were mapped', f'q1: mode {q_modes[0]},', f'q2: mode {q_modes[1]},', f'q3: mode {q_modes[2]},', f'q4: mode {q_modes[3]}')
# old_evecs = evecs.copy()
# for i in range(4):
#     evecs[:,i] = old_evecs[:,q_modes[i]]
# # evals = np.array([evals[q_modes[i]] for i in range(4)])
# ECs = qe**2/2/h * evecs.transpose() @ inv_c_matrix @ evecs
# EJs = flux_quant**2/h/4/np.pi**2 * evecs.transpose() @ inv_l_matrix @ evecs
# ===============================================

ECs = qe**2/2/h * inv_c_matrix
EJs = flux_quant**2/h/4/np.pi**2 * inv_l_matrix
Zs = np.sqrt(8*np.diag(ECs)/np.diag(EJs))
gs = np.array([[4*ECs[i,j]/np.sqrt(Zs[i]*Zs[j]) for i in range(len(c_matrix))] for j in range(len(c_matrix))])

print('all ECs\n', 1e-9*ECs)
print('all EJs\n', 1e-9*ECs)
print('all q-q gs\n', 1e-6*gs)
print('impedances (unitless)\n', *Zs)

ECs = 1e-9 * np.diag(ECs)
EJs = 1e-9 * np.diag(EJs)

gs = 1e-9 * np.abs(np.array([gs[0,1]])) # g12, g23, g24

# AFTER HERE EVERYTHING IS INDEXED SO QUBIT NUMBER = INDEX AND EVERYTHING IS IN GHz

Modes (cols are evecs)
 [[-0.989  0.146]
 [ 0.146  0.989]]
all ECs
 [[0.19006303 0.00587122]
 [0.00587122 0.22889316]]
all EJs
 [[0.19006303 0.00587122]
 [0.00587122 0.22889316]]
all q-q gs
 [[1904.31159593   60.10340605]
 [  60.10340605 2394.05195711]]
impedances (unitless)
 0.39922674967106897 0.38243641487447083


In [75]:
cutoffs = [5,5]
transmon1 = scq.Transmon(EC=ECs[0], EJ=EJs[0], ng=0, ncut=110, truncated_dim=cutoffs[0])
transmon2 = scq.Transmon(EC=ECs[1], EJ=EJs[1], ng=0, ncut=110, truncated_dim=cutoffs[1])

evals1, evecs1 = transmon1.eigensys(evals_count=cutoffs[0])
evals2, evecs2 = transmon2.eigensys(evals_count=cutoffs[1])

evals1 -= evals1[0]
evals2 -= evals2[0]

alpha1 = evals1[2]-2*evals1[1]
alpha2 = evals2[2]-2*evals2[1]

qubit_freqs = np.array([evals1[1], evals2[1]])
alphas = np.array([alpha1, alpha2])

charge_mat_el = 1/np.sqrt(2) * (EJs/8/ECs)**1/4
g_rqs = Crqs/(Crqs+Cjs) * res_freqs * np.sqrt(2*Zo/(hbar/qe**2))
deltas = qubit_freqs-res_freqs
lamb_shifts = g_rqs**2/deltas
chi_shifts = g_rqs**2/deltas * alphas/(deltas+alphas)
drive_purcell_rate_analytical = (2*np.pi*qubit_freqs*1e9*Cds)**2/(Cjs+Cds) * 50 /2/np.pi
drive_purcell_rate_sim = qubit_freqs*1e9/Q_drives
kappas = res_freqs/Q_res
res_purcell_rate = kappas * (g_01s/deltas)**2

print('EC values (GHz):\n\t', *ECs)
print('EJ values (GHz):\n\t', *EJs)
print('qubit freqs/2pi (GHz):\n\t', *qubit_freqs)
print('alphas/2pi (GHz):\n\t', *alphas)
print('q-q couplings g12, g23, g24 (MHz)\n\t', *(gs*1e3))
print('q-res couplings (MHz) [want 50-100]:\n\t', *(1e3*g_rqs))
print('delta wq-wr (GHz) [want 1.5-2]:\n\t', *deltas)
print('lamb shifts (MHz):\n\t', *(1e3*lamb_shifts))
print('chi shifts (MHz) [want 0.15-0.5]:\n\t', *(1e3*chi_shifts))
print('\n')
print('drive purcell limits - analytical (ms) [want >1 ms]:\n\t', *(1e3/drive_purcell_rate_analytical))
print('drive purcell limits - simulated (ms) [want >1 ms]:\n\t', *(1e3/drive_purcell_rate_sim))
print('res linewidth kappas - simulated (kHz):\n\t', *(1e6*kappas))
print('res purcell limits - simulated (s):\n\t', *(1/res_purcell_rate))

EC values (GHz):
	 0.19006303220111473 0.22889316187549003
EJ values (GHz):
	 9.540000000000003 12.52
qubit freqs/2pi (GHz):
	 3.6075460265547523 4.546593968554438
alphas/2pi (GHz):
	 -0.2183470938636214 -0.26098169191327614
q-q couplings g12, g23, g24 (MHz)
	 60.10340605249436
q-res couplings (MHz) [want 50-100]:
	 80.93377197396795 88.05689523601055
delta wq-wr (GHz) [want 1.5-2]:
	 -1.792453973445248 -1.8534060314455623
lamb shifts (MHz):
	 -3.654361865339313 -4.183657907144069
chi shifts (MHz) [want 0.15-0.5]:
	 -0.39681662507309434 -0.5163944658448592


drive purcell limits - analytical (ms) [want >1 ms]:
	 5.99192657948907 4.069499435236433
drive purcell limits - simulated (ms) [want >1 ms]:
	 inf inf
res linewidth kappas - simulated (kHz):
	 0.0 0.0
res purcell limits - simulated (s):
	 inf inf


 /Users/conniemiao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py: 38RuntimeWarning: divide by zero encountered in true_divide
 /Users/conniemiao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py: 40